# STM32 Pi Interface: REPL & GPIO

`/boot/config.txt`: add

```
[pi4]
dtoverlay=uart3
dtoverlay=uart4
enable_uart=0
```

Overlay `enable_uart=0` disables the console UART on pins 8, 10. No need to disable if pins are not connected (but then there is no way this can work with a Pi 3, probably ok).

UART Assignments (Pi 4):

```
Pi 4                            STM32
/dev/ttyAMA1   uart3            uart6 (REPL)
/dev/ttyAMA2   uart4            uart3 (dfu, robot comm)
```

GPIO Pins:

```
BOOT0   = 13 (GPIO27)
SHUT_DN = 33 (GPIO13)     shutdown button input (active low)
nRST    = 40 (GPIO21)
AUX     = 36 (GPIO16)     stm32: AUX
                          stm32: PWR_EN: pull low to cut main (5V) power supply
                          stm32: V12_DIV: 12V input / 7.8 
```

## IoT Kernel

In [1]:
%connect 'serial:///dev/ttyAMA1'

Connected to stm32 @ serial:///dev/ttyAMA1


In [1]:
for i in range(5):
    print(i**7)

0
1
128
2187
16384


## Reset, Exec

In [2]:
%%host

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

print(">>> reset example:")

stm32.exec("a = 2**8")
print(stm32.exec("print('before reset: a =', a)"))
stm32.hard_reset()
try:
    print(stm32.exec("print('after  reset:', a)"))
except stm32.RemoteError as e:
    print("after reset:")
    print(e)
    
print("\n\n>>> data_consumer example:")
    
code = """
import time

for i in range(9):
    print(i**4)
    time.sleep(0.2)
"""
stm32.exec(code, data_consumer=lambda x: print(x.decode(), end=""))

print("\n\n>>> exec_no_follow example:")
    
stm32.exec_no_follow("print('going for a nap'); import time; time.sleep(3600)")
print("control returns immediately to pi")

>>> reset example:
before reset: a = 256

after reset:
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'a' isn't defined



>>> data_consumer example:
0
1
16
81
256
625
1296
2401
4096


>>> exec_no_follow example:
*** MCU: going for a nap

control returns immediately to pi


In [2]:
%%host

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

stm32.exec_no_follow("print('going for a nap'); import time; time.sleep(3600)")
print("control returns immediately to pi")

*** MCU: going for a nap

control returns immediately to pi


## Rsync

In [5]:
%rlist

%%host

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

stm32.rsync(dry_run=True)

                            flash/
    372  Apr 28 01:04 2021      boot.py
                                lib/
   6644  Apr 23 09:18 2021          bdevice.py
   6731  Apr 23 09:18 2021          flash_spi.py
                            spi/
                                lib/
   6491  Apr 01 12:55 2021          bno055.py
   5598  Apr 23 17:58 2021          bno055_base.py
   1078  Apr 01 12:55 2021          bno055_test.py
   4537  Apr 23 16:55 2021          comm.py
   4742  Apr 23 18:06 2021          controller.py
   2073  Apr 01 12:55 2021          encoder.py
    244  Apr 01 12:55 2021          init_urpc_client.py
   2094  Apr 01 12:55 2021          logging.py
   1174  Apr 01 12:55 2021          neopixel.py
   2599  Apr 23 09:42 2021          pid.py
   7813  Apr 01 12:55 2021          pystone.py
    317  Apr 01 12:55 2021          secrets.py
    978  Apr 01 12:55 2021          tb6612.py
  22299  Apr 01 12:55 2021          vl53l0x.py
   7376  Apr 01 12:55 2021          vl53l1x.py
UPDAT

## Supply Voltage

In [4]:
%%host

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

stm32.supply_voltage()

12.0497

## Power-off

In [ ]:
%%host

############
# MUST BE RUN FROM Python3 kernel!

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

stm32.power_off()

## SHUT_DN Button

In [3]:
%%host

from gpiozero import Button

def shut_down_pi():
    # shut down and power off pi
    pass

with Button(13, pull_up=True, bounce_time=0.1) as shut_dn:

    def say_pressed():
        print("pressed")
        
    shut_dn.when_pressed = say_pressed 

    import time
    time.sleep(120)

pressed
released
pressed
released
pressed
released
pressed
released
pressed
released


In [ ]:
%%bash

cd ~/iot49

# activate virtual environment from script
eval "$(direnv export bash)"

python ~/iot49/projects/robot/code-pi/shutdown_monitor.py

## Flash MicroPython

In [7]:
%%host

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

import stm32

stm32.flash_bin(info_only=True)
# stm32.flash()

stm32flash 0.6

http://stm32flash.sourceforge.net/

Interface serial_posix: 57600 8E1
Version      : 0x31
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0413 (STM32F40xxx/41xxx)
- RAM        : Up to 128KiB  (12288b reserved by bootloader)
- Flash      : Up to 1024KiB (size first sector: 1x16384)
- Option RAM : 16b
- System RAM : 30KiB


